In [22]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17080, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 17080 (delta 20), reused 7 (delta 7), pack-reused 17049 (from 2)
Receiving objects: 100% (17080/17080), 15.68 MiB | 28.78 MiB/s, done.
Resolving deltas: 100% (11720/11720), done.


In [23]:
%cd yolov5
!pip install -r requirements.txt

/kaggle/working/yolov5/yolov5/yolov5/yolov5


In [24]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)  

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-27 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15095MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 152MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [25]:
!pip install effdet

In [26]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision import datasets
from tqdm import tqdm
from PIL import Image
import os
from effdet import create_model as EfficientDet
from torchvision.transforms import Resize
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="torch.cuda.amp")

class GradientReversalLayer(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.alpha * grad_output, None

class UniversalDomainGeneralizationModule(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.feature_size = None  
        self.domain_discriminator = None  

    def forward(self, x, domain_label=None, alpha=1.0):
        if isinstance(x, list):
            x = torch.stack(x, dim=0) 
        
        features = self.backbone(x)
        if isinstance(features, tuple) or isinstance(features, list):
            combined_features = torch.cat([feat.flatten(1) for feat in features], dim=1)
        elif isinstance(features, dict):
            combined_features = torch.cat([feat.flatten(1) for feat in features.values()], dim=1)
        else:
            combined_features = features.flatten(1)

        if self.feature_size is None:
            self.feature_size = combined_features.size(1)
            self.domain_discriminator = nn.Sequential(
                nn.Linear(self.feature_size, 128),
                nn.ReLU(),
                nn.Linear(128, 1),
                nn.Sigmoid()
            )
            self.domain_discriminator.to(combined_features.device)  

        if domain_label is not None:
            reversed_features = GradientReversalLayer.apply(combined_features, alpha)
            domain_preds = self.domain_discriminator(reversed_features)
            return combined_features, domain_preds
        return combined_features, None


class FasterRCNNWithUDGM(nn.Module):
    def __init__(self):
        super().__init__()
        self.faster_rcnn = fasterrcnn_resnet50_fpn(pretrained=True)
        self.udgm = UniversalDomainGeneralizationModule(self.faster_rcnn.backbone)

    def forward(self, images, targets=None, domain_label=None, alpha=1.0):
        if isinstance(images, list):
            stacked_images = torch.stack(images, dim=0)
        else:
            stacked_images = images

        features, domain_preds = self.udgm(stacked_images, domain_label, alpha) 

        if targets is not None:
            outputs = self.faster_rcnn(images, targets)
            return outputs, domain_preds
        outputs = self.faster_rcnn(images)
        return outputs, domain_preds

class YOLOv5WithUDGM(nn.Module):
    def __init__(self):
        super().__init__()
        self.yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        
        self.backbone = self.yolov5.model.model
        
        self.udgm = UniversalDomainGeneralizationModule(self.backbone)

    def forward(self, images, targets=None, domain_label=None, alpha=1.0):
        if isinstance(images, list):
            images = torch.stack(images, dim=0)

        features, domain_preds = self.udgm(images, domain_label, alpha)

        predictions = self.yolov5(images)

        if targets is not None:
            return predictions, domain_preds
        return predictions, domain_preds

import torch.nn.functional as F

class EfficientDetWithUDGM(nn.Module):
    def __init__(self):
        super().__init__()
        self.effdet = EfficientDet('tf_efficientdet_d0', pretrained=True, num_classes=20)
        self.udgm = UniversalDomainGeneralizationModule(self.effdet.backbone)

    def forward(self, images, targets=None, domain_label=None, alpha=1.0):
        if isinstance(images, list):
            images = torch.stack(images, dim=0)

        features, domain_preds = self.udgm(images, domain_label, alpha)
        predictions = self.effdet(images)

        if isinstance(predictions, tuple) and len(predictions) == 2:
            pred_classes, pred_boxes = predictions
            pred_classes, pred_boxes = _fix_prediction_shapes(pred_classes, pred_boxes, images.device)  
            return (pred_classes, pred_boxes), domain_preds
        else:
            raise ValueError(f"Unexpected EfficientDet predictions format: {predictions}")

def _fix_prediction_shapes(pred_classes, pred_boxes, device):
    if not isinstance(pred_classes, list):
        pred_classes = [pred_classes]
    if not isinstance(pred_boxes, list):
        pred_boxes = [pred_boxes]

    max_h = max(tensor.size(2) for tensor in pred_classes)
    max_w = max(tensor.size(3) for tensor in pred_classes)

    pred_classes = torch.cat([
        F.pad(
            tensor, (0, max_w - tensor.size(3), 0, max_h - tensor.size(2)), value=0
        ).to(device)
        for tensor in pred_classes
    ], dim=1)  

    pred_boxes = torch.cat([
        F.pad(
            tensor, (0, max_w - tensor.size(3), 0, max_h - tensor.size(2)), value=0
        ).to(device)
        for tensor in pred_boxes
    ], dim=1) 

    return pred_classes, pred_boxes


class CityscapesDataset(torch.utils.data.Dataset):
    def __init__(self, root, split="train", transforms=None):
        self.root = root
        self.split = split
        self.transforms = transforms
        self.img_dir = os.path.join(root, split, "img")
        self.label_dir = os.path.join(root, split, "label")
        self.imgs = sorted(os.listdir(self.img_dir))
        self.labels = sorted(os.listdir(self.label_dir))

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.imgs[idx])
        label_path = os.path.join(self.label_dir, self.labels[idx])
        img = Image.open(img_path).convert("RGB")
        label = Image.open(label_path)

        if self.transforms:
            img = self.transforms(img)

        boxes = torch.tensor([[0, 0, 10, 10]]) 
        labels = torch.tensor([1]) 
        targets = {"boxes": boxes, "labels": labels}

        return img, targets

from torchvision.transforms import Compose, Resize, Normalize
from torchvision.transforms.functional import to_tensor

def get_cityscapes_data_loader(root, split="train", batch_size=4, model_type="fasterrcnn"):
    if model_type == "fasterrcnn":
        transforms = lambda x: to_tensor(x)
    elif model_type == "efficientdet":
        transforms = Compose([
            Resize((512, 512)),
            to_tensor,
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    elif model_type == "yolov5":
        transforms = Compose([
            Resize((640, 640)),
            ToTensor(),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
        ])
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

    dataset = CityscapesDataset(root, split=split, transforms=transforms)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


def kl_divergence(p, q):
    p = p.clamp(min=1e-6)
    q = q.clamp(min=1e-6)
    return (p * (p.log() - q.log())).sum()

def iou_regularization(pred_boxes, true_boxes):
    iou = compute_iou(pred_boxes, true_boxes)
    return 1 - iou.mean()

def compute_iou(pred_boxes, true_boxes):
    inter_xmin = torch.max(pred_boxes[:, 0], true_boxes[:, 0])
    inter_ymin = torch.max(pred_boxes[:, 1], true_boxes[:, 1])
    inter_xmax = torch.min(pred_boxes[:, 2], true_boxes[:, 2])
    inter_ymax = torch.min(pred_boxes[:, 3], true_boxes[:, 3])

    inter_area = (inter_xmax - inter_xmin).clamp(0) * (inter_ymax - inter_ymin).clamp(0)
    pred_area = (pred_boxes[:, 2] - pred_boxes[:, 0]) * (pred_boxes[:, 3] - pred_boxes[:, 1])
    true_area = (true_boxes[:, 2] - true_boxes[:, 0]) * (true_boxes[:, 3] - true_boxes[:, 1])

    union_area = pred_area + true_area - inter_area
    iou = inter_area / union_area.clamp(min=1e-6)
    return iou

def evaluate(model, data_loader, device, model_type="fasterrcnn"):
    model.eval()
    all_iou = []
    total_predictions = 0
    correct_predictions = 0

    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Evaluation Loop"):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            if model_type == "fasterrcnn":
                outputs, _ = model(images, domain_label=None) 
                for i, output in enumerate(outputs):
                    pred_boxes = output['boxes']
                    true_boxes = targets[i]['boxes']

                    iou = compute_iou(pred_boxes, true_boxes)
                    all_iou.append(iou.mean().item())

                    pred_labels = output['labels']
                    true_labels = targets[i]['labels']

                    total_predictions += len(pred_labels)
                    correct_predictions += (pred_labels == true_labels).sum().item()
                    
            elif model_type == "yolov5":
                images_np = [img.permute(1, 2, 0).cpu().numpy() for img in images]  
                outputs = model.yolov5(images_np)
                
                for i, output in enumerate(outputs.xyxy): 
                    if output is None or output.shape[0] == 0:  
                        continue

                    pred_boxes = output[:, :4]  
                    pred_labels = output[:, -1].long() 
                    true_boxes = targets[i]["boxes"]
                    true_labels = targets[i]["labels"]

                    iou = compute_iou(pred_boxes, true_boxes)
                    all_iou.append(iou.mean().item())

                    total_predictions += len(pred_labels)
                    correct_predictions += (pred_labels == true_labels).sum().item()
                    
            elif model_type == "efficientdet":
                    outputs, _ = model(images, domain_label=None)  
                    if isinstance(outputs, tuple):  
                        pred_classes, pred_boxes = outputs
                        pred_classes, pred_boxes = _fix_prediction_shapes(pred_classes, pred_boxes, device)

                        for i, target in enumerate(targets):
                            gt_boxes = target['boxes']
                            gt_labels = target['labels']

                            pred_boxes = pred_boxes.view(-1, 4)
                            pred_classes = pred_classes.view(-1, pred_classes.size(-1)).argmax(dim=1)

                            if pred_boxes.size(0) > 0 and gt_boxes.size(0) > 0:
                                iou = compute_iou(pred_boxes, gt_boxes)
                                all_iou.append(iou.mean().item())
                                total_predictions += pred_classes.size(0)
                                correct_predictions += (pred_classes == gt_labels).sum().item()
            else:
                raise ValueError(f"Unsupported model type: {model_type}")

    mean_iou = sum(all_iou) / len(all_iou) if all_iou else 0.0
    mAP = correct_predictions / total_predictions if total_predictions else 0.0
    wada = (mean_iou + mAP) / 2  

    return mAP, wada, mean_iou


def train(model, train_loader, optimizer, criterion, device):
    model.train()
    accumulation_steps = 2
    optimizer.zero_grad()

    for i, (images, targets) in enumerate(tqdm(train_loader, desc="Training Loop")):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        outputs, domain_preds = model(images, targets, domain_label=torch.ones(len(images), device=device))

        if hasattr(model, 'faster_rcnn'):
            if isinstance(outputs, dict):
                detection_loss = sum(outputs.values())
            else:
                raise ValueError(f"[DEBUG] Unexpected Faster R-CNN outputs type: {type(outputs)}")

        elif hasattr(model, 'effdet'):
            if isinstance(outputs, tuple) and len(outputs) == 2:
                detection_loss = compute_efficientdet_loss(outputs, targets)
            else:
                raise ValueError(f"[DEBUG] Unexpected EfficientDet outputs format: {outputs}")

        elif hasattr(model, 'yolov5'):
            if isinstance(outputs, list):
                detection_loss = compute_yolov5_loss(outputs, targets)
            else:
                raise ValueError(f"[DEBUG] Unexpected YOLOv5 outputs format: {outputs}")

        else:
            raise ValueError("[DEBUG] Model type not supported.")

        kl_loss = kl_divergence(domain_preds, torch.ones_like(domain_preds))
        iou_loss = iou_regularization(targets[0]['boxes'], targets[0]['boxes'])

        total_loss = detection_loss + kl_loss + iou_loss
        total_loss = total_loss / accumulation_steps
        total_loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

def compute_yolov5_loss(outputs, targets):
    cls_loss = 0.0
    box_loss = 0.0

    for output, target in zip(outputs.xyxy, targets):  
        if output is None or output.shape[0] == 0: 
            continue

        pred_boxes = output[:, :4] 
        pred_scores = output[:, 4] 
        pred_classes = output[:, 5:].argmax(dim=1)  

        gt_boxes = target['boxes']
        gt_labels = target['labels']

        iou_matrix = compute_iou_matrix(pred_boxes, gt_boxes)
        max_iou, matched_indices = iou_matrix.max(dim=1)

        positive_indices = max_iou > 0.5
        if positive_indices.sum() > 0:
            pred_boxes_pos = pred_boxes[positive_indices]
            gt_boxes_pos = gt_boxes[matched_indices[positive_indices]]
            box_loss += nn.SmoothL1Loss()(pred_boxes_pos, gt_boxes_pos)

        cls_loss += focal_loss(pred_classes.float(), gt_labels.float())

    total_loss = cls_loss + box_loss
    return total_loss


def compute_efficientdet_loss(predictions, targets, num_classes=20):
    cls_loss = 0.0
    box_loss = 0.0

    pred_classes, pred_boxes = predictions
   
    for pred_cls, pred_box, target in zip(pred_classes, pred_boxes, targets):
        gt_boxes = target['boxes']
        gt_labels = target['labels']

        matched_cls_targets, matched_box_targets, positive_indices = generate_anchor_targets(
            pred_box.view(-1, 4), gt_boxes, gt_labels, num_classes
        )
        matched_cls_targets = matched_cls_targets.view_as(pred_cls)

        pred_cls = pred_cls.view(-1, num_classes)  
        matched_cls_targets = matched_cls_targets.view(-1, num_classes)

        cls_loss += focal_loss(pred_cls, matched_cls_targets)

        if positive_indices.sum() > 0:
            pred_boxes_pos = pred_box.view(-1, 4)[positive_indices]
            matched_boxes_pos = matched_box_targets[positive_indices]
            box_loss += nn.SmoothL1Loss()(pred_boxes_pos, matched_boxes_pos)

    total_loss = (cls_loss + box_loss) / len(targets)
    return total_loss

def generate_anchor_targets(pred_boxes, gt_boxes, gt_labels, num_classes):
    if not isinstance(pred_boxes, torch.Tensor):
        pred_boxes = torch.tensor(pred_boxes, device=gt_boxes.device)
    if not isinstance(gt_boxes, torch.Tensor):
        gt_boxes = torch.tensor(gt_boxes, device=pred_boxes.device)

    iou_matrix = compute_iou_matrix(pred_boxes, gt_boxes)
    max_iou, matched_indices = iou_matrix.max(dim=1)

    positive_indices = max_iou > 0.5
    negative_indices = max_iou < 0.4

    matched_cls_targets = torch.zeros((pred_boxes.size(0), num_classes), device=pred_boxes.device)

    if positive_indices.any():
        matched_gt_labels = gt_labels[matched_indices[positive_indices]].long()
        matched_cls_targets[positive_indices, matched_gt_labels] = 1.0

    matched_box_targets = torch.zeros_like(pred_boxes)
    if positive_indices.any():
        matched_box_targets[positive_indices] = gt_boxes[matched_indices[positive_indices]]

    return matched_cls_targets, matched_box_targets, positive_indices

def compute_iou_matrix(pred_boxes, gt_boxes):
    inter_xmin = torch.max(pred_boxes[:, None, 0], gt_boxes[:, 0])
    inter_ymin = torch.max(pred_boxes[:, None, 1], gt_boxes[:, 1])
    inter_xmax = torch.min(pred_boxes[:, None, 2], gt_boxes[:, 2])
    inter_ymax = torch.min(pred_boxes[:, None, 3], gt_boxes[:, 3])

    inter_area = (inter_xmax - inter_xmin).clamp(0) * (inter_ymax - inter_ymin).clamp(0)
    pred_area = (pred_boxes[:, 2] - pred_boxes[:, 0]) * (pred_boxes[:, 3] - pred_boxes[:, 1])
    gt_area = (gt_boxes[:, 2] - gt_boxes[:, 0]) * (gt_boxes[:, 3] - gt_boxes[:, 1])

    union_area = pred_area[:, None] + gt_area - inter_area
    iou_matrix = inter_area / union_area.clamp(min=1e-6)
    return iou_matrix

def focal_loss(pred_cls, targets, alpha=0.25, gamma=2.0):
    probs = pred_cls.sigmoid()
    ce_loss = nn.BCEWithLogitsLoss(reduction="none")(pred_cls, targets)
    pt = torch.where(targets == 1, probs, 1 - probs)  
    focal_loss = alpha * (1 - pt) ** gamma * ce_loss
    return focal_loss.sum()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = "fasterrcnn"  

if model_type == "fasterrcnn":
    model = FasterRCNNWithUDGM().to(device)
elif model_type == "yolov5":
    model = YOLOv5WithUDGM().to(device)
elif model_type == "efficientdet":
    model = EfficientDetWithUDGM().to(device)
else:
    raise ValueError(f"Unsupported model type: {model_type}")

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

dataset_path = "/kaggle/input/cityscapes"
train_loader = get_cityscapes_data_loader(dataset_path, split="train", batch_size=4, model_type="fasterrcnn")
val_loader = get_cityscapes_data_loader(dataset_path, split="val")

if model_type=="fasterrcnn":
    for epoch in range(1):
        print(f"Epoch {epoch+1}")
        train(model, train_loader, optimizer, criterion, device)

    mAP, wada, iou = evaluate(model, val_loader, device)
    print(f"Evaluation Results: mAP={mAP:.4f}, WADA={wada:.4f}, IoU={iou:.4f}")
else:
    pass
torch.save(model.state_dict(), "/kaggle/working/udgm_model.pth")

Epoch 1


Evaluation Loop: 100%|██████████| 125/125 [00:54<00:00,  2.27it/s]


Evaluation Results: mAP=1.0000, WADA=0.9914, IoU=0.9828
